In [1]:
from scipy.interpolate import interp1d
from scipy.integrate import simpson
import numpy as np

# 设计船与母船尺度数据

In [2]:
L_total_parent = 225 # 母船总长度
Lpp_parent = 218 # 母船垂线间长
B_parent = 32.26 # 母船型宽
D_parent = 19.6 # 母船型深
d_design_parent = 12.5 # 母船设计吃水
d_structure_parent = 14.3 # 母船结构吃水\
DW_parent = 75088 # 母船载重量 :t
Lw_parent = Lpp_parent+5*0.75 # 水线间长:m ,首垂线与尾垂线距离+ 尾垂线后5个肋位，每个肋位距离为0.75m，参考母船总布置图
L_ball_parent = 4*0.75 # 首垂线至球首前端距，#256 - # 260，肋距为0.75m，参考母船总布置图
nabla_parent = 85488.56 # 母船结构吃水14.3m处的排水体积，参考静水力计算书
delta_parent = 87809.16 # 母船结构吃水处的排水量,参考静水力计算书吃水14.3m时的displt

In [3]:
# 设计船信息
DW = 72100 # 设计船载重量,t
Cb = 0.85
Lpp = 215.17    # m
B = 31.84       # m 
D = 19.34       # m
d = 14.11       # m
delta = 84292   # t

# 截面面积

In [4]:
# No.1货舱范围17-19站
A_17_stand = 306.61 * 2 # 17站截面的面积
A_1705_stand = 294.93 * 2 # 17.5站截面的面积
A_18_stand = 276.11 * 2 # 18站
A_1805_stand = 242.77 * 2 # 18.5站
A_19_stand = 196.86 * 2 # 19站
# No.2货舱范围
A_16_stand = 314.38 * 2 # 16站
A_15_stand = 311.30 * 2 # 15站
A_14_stand = 309.27 * 2 # 14站
A_13_stand = 309.27 * 2 # 13站
A_12_stand = 309.27 * 2
A_11_stand = 309.27 * 2
A_10_stand = 309.27 * 2
A_9_stand = 309.27 * 2
A_8_stand = 309.27 * 2 #平行中体的位置

A_7_stand = 308.64 * 2 # 7站
A_605_stand = 308.62 * 2 # 6.5站
A_6_stand = 311.89 * 2 # 6站
A_505_stand = 311.41 * 2 # 5.5站
A_5_stand = 308.59 * 2 # 5站
A_405_stand = 303.41 * 2 # 4.5站
A_4_stand = 294.87 * 2 # 4站
A_305_stand = 282.17 * 2 # 3.5站
A_3_stand = 246.49 * 2 # 3站
A_205_stand = 246.50 * 2 # 2.5站
A_2_stand = 225.26 * 2 # 2站
A_105_stand = 191.21 * 2 # 1.5站
A_1_stand = 152.21 * 2 # 1站
A_075_stand = 129.23 * 2 # 0.75站
A_05_stand = 103.59 * 2 # 0.5站
A_0_stand = (72.45 + 0.82) * 2 # 0站,0.82是螺旋桨

A_tail = 52.82 * 2 # 艉封板
A_side = 13.80 * 2 # 侧边舱
A_double_deck = 39.12 # 双层底
A_top = 19.88 * 2 # 顶边舱
A_bottom = 15.24 # 底边舱

# 货舱

## 母船货舱数据

In [5]:
# 容积
V_cargo1_parent = 10428.53 # m3
V_cargo2_parent = 13612.98
V_cargo3_parent = 13477.53
V_cargo4_parent = 13482.34
V_cargo5_parent = 13571.75
V_cargo6_parent = 13566.95
V_cargo7_parent = 12117.57
# 重心在货舱中的齐次坐标: (重心坐标 - 货舱起始坐标)/货舱长度 (重心坐标 - 货舱起始高度)/货舱高度 横向坐标统一为0
LCG_ratio_cargo1_parent = (194.613 - 183.98)/23.49
LCG_ratio_cargo2_parent = (170.559 - 157.88)/25.23
LCG_ratio_cargo3_parent = (144.513 - 131.78)/25.23
LCG_ratio_cargo4_parent = (118.418 - 105.68)/25.23
LCG_ratio_cargo5_parent = (92.236 - 79.58)/25.23
LCG_ratio_cargo6_parent = (66.130 - 53.48)/25.23
LCG_ratio_cargo7_parent = (40.993 - 28.25)/25.36

VCG_ratio_cargo1_parent = (11.481 - 1.71)/17.89
VCG_ratio_cargo2_parent = (11.084 - 1.71)/17.89
VCG_ratio_cargo3_parent = (11.046 - 1.71)/17.89
VCG_ratio_cargo4_parent = (11.047 - 1.71)/17.89
VCG_ratio_cargo5_parent = (11.052 - 1.71)/17.89
VCG_ratio_cargo6_parent = (11.051 - 1.71)/17.89
VCG_ratio_cargo7_parent = (11.489 - 1.71)/17.89

## 舱容估算

In [6]:
# 母船缩放
k = D * B * Lpp / (D_parent*B_parent*Lpp_parent)
V_cargo1 = V_cargo1_parent * k
V_cargo2 = V_cargo2_parent * k
V_cargo3 = V_cargo3_parent * k
V_cargo4 = V_cargo4_parent * k
V_cargo5 = V_cargo5_parent * k
V_cargo6 = V_cargo6_parent * k
V_cargo7 = V_cargo7_parent * k
print("""
第一货舱的容积为  %.2f m3
第二货舱的容积为  %.2f m3
第三货舱的容积为  %.2f m3
第四货舱的容积为  %.2f m3
第五货舱的容积为  %.2f m3
第六货舱的容积为  %.2f m3
第七货舱的容积为  %.2f m3
"""%(V_cargo1, V_cargo2, V_cargo3, V_cargo4, V_cargo5, V_cargo6, V_cargo7))


第一货舱的容积为  10024.38 m3
第二货舱的容积为  13085.42 m3
第三货舱的容积为  12955.22 m3
第四货舱的容积为  12959.84 m3
第五货舱的容积为  13045.78 m3
第六货舱的容积为  13041.17 m3
第七货舱的容积为  11647.96 m3



## 重心估算

In [7]:
L_cargo1 = L_cargo2 = L_cargo3 = L_cargo4 = L_cargo5 = L_cargo6 = 24.65
L_cargo7 = 23.8
start_cargo1 = 179.6
start_cargo2 = 154.1
start_cargo3 = 128.6
start_cargo4 = 103.1
start_cargo5 = 77.6
start_cargo6 = 52.1
start_cargo7 = 27.45
start_v_cargo = 1.684 # 双层底高度
Height_cargo = 17.657
LCG_cargo1 = start_cargo1 + L_cargo1 * LCG_ratio_cargo1_parent
LCG_cargo2 = start_cargo2 + L_cargo2 * LCG_ratio_cargo2_parent
LCG_cargo3 = start_cargo3 + L_cargo3 * LCG_ratio_cargo3_parent
LCG_cargo4 = start_cargo4 + L_cargo4 * LCG_ratio_cargo4_parent
LCG_cargo5 = start_cargo5 + L_cargo5 * LCG_ratio_cargo5_parent
LCG_cargo6 = start_cargo6 + L_cargo6 * LCG_ratio_cargo6_parent
LCG_cargo7 = start_cargo7 + L_cargo7 * LCG_ratio_cargo7_parent

VCG_cargo1 = start_v_cargo + Height_cargo * VCG_ratio_cargo1_parent
VCG_cargo2 = start_v_cargo + Height_cargo * VCG_ratio_cargo2_parent
VCG_cargo3 = start_v_cargo + Height_cargo * VCG_ratio_cargo3_parent
VCG_cargo4 = start_v_cargo + Height_cargo * VCG_ratio_cargo4_parent
VCG_cargo5 = start_v_cargo + Height_cargo * VCG_ratio_cargo5_parent
VCG_cargo6 = start_v_cargo + Height_cargo * VCG_ratio_cargo6_parent
VCG_cargo7 = start_v_cargo + Height_cargo * VCG_ratio_cargo7_parent
print("""
第一货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
第二货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
第三货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
第四货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
第五货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
第六货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
第七货舱的重心纵向垂向坐标为  (%.2f, %.2f) m3
"""%(LCG_cargo1,VCG_cargo1,LCG_cargo2,VCG_cargo2,
    LCG_cargo3,VCG_cargo3,LCG_cargo4,VCG_cargo4,
    LCG_cargo5,VCG_cargo5,LCG_cargo6,VCG_cargo6,
    LCG_cargo7,VCG_cargo7))


第一货舱的重心纵向垂向坐标为  (190.76, 11.33) m3
第二货舱的重心纵向垂向坐标为  (166.49, 10.94) m3
第三货舱的重心纵向垂向坐标为  (141.04, 10.90) m3
第四货舱的重心纵向垂向坐标为  (115.55, 10.90) m3
第五货舱的重心纵向垂向坐标为  (89.97, 10.90) m3
第六货舱的重心纵向垂向坐标为  (64.46, 10.90) m3
第七货舱的重心纵向垂向坐标为  (39.41, 11.34) m3



## 重量计算

In [9]:
# 货物重量
rho = 1/1.26 # 参考书p33也货物的积载系数，V=W * u，u是积载因数，因此rho=W/V=1/u
print("货物密度为%.3f t/m^3"%rho)

货物密度为0.794 t/m^3


# 压载水舱

## 侧压载舱

In [8]:
W,H,L = 1.283,10.758,25.5 # 平行中体部分侧压载舱S的尺寸
A1 = 30.678 # 第一压载舱S俯视图面积
H1 = 10.758 # 第一压载舱S高度
A2 = 29.826 # 第七压载舱S俯视图面积
H2 = 9.613 # 第七压载舱S的上半部分高度
ratio = 48.386 / 228.31 # 第七压载舱S下半与上半侧面积之比，这里认为下半与上半压载舱体积之比等于其侧面积之比
# 体积计算
V_SBWT2 = V_SBWT3 = V_SBWT4 = V_SBWT5 = V_SBWT6 = W * H * L # 单侧压载舱的体积
V_SBWT1 = A1 * H1 # 单侧
V_SBWT7 = A2 * H2 * (1 + ratio)
print("""
第一侧压载舱S的容积为  %.2f m3
第一侧压载舱P的容积为  %.2f m3
第二侧压载舱S的容积为  %.2f m3
第二侧压载舱P的容积为  %.2f m3
第三侧压载舱S的容积为  %.2f m3
第三侧压载舱P的容积为  %.2f m3
第四侧压载舱S的容积为  %.2f m3
第四侧压载舱P的容积为  %.2f m3
第五侧压载舱S的容积为  %.2f m3
第五侧压载舱P的容积为  %.2f m3
第六侧压载舱S的容积为  %.2f m3
第六侧压载舱P的容积为  %.2f m3
第七侧压载舱S的容积为  %.2f m3
第七侧压载舱P的容积为  %.2f m3
"""%(V_SBWT1,V_SBWT1,V_SBWT2,V_SBWT2,
    V_SBWT3,V_SBWT3,V_SBWT4,V_SBWT4,
    V_SBWT5,V_SBWT5,V_SBWT6,V_SBWT6,
    V_SBWT7,V_SBWT7))


第一侧压载舱S的容积为  330.03 m3
第一侧压载舱P的容积为  330.03 m3
第二侧压载舱S的容积为  351.96 m3
第二侧压载舱P的容积为  351.96 m3
第三侧压载舱S的容积为  351.96 m3
第三侧压载舱P的容积为  351.96 m3
第四侧压载舱S的容积为  351.96 m3
第四侧压载舱P的容积为  351.96 m3
第五侧压载舱S的容积为  351.96 m3
第五侧压载舱P的容积为  351.96 m3
第六侧压载舱S的容积为  351.96 m3
第六侧压载舱P的容积为  351.96 m3
第七侧压载舱S的容积为  347.48 m3
第七侧压载舱P的容积为  347.48 m3



In [9]:
# 平行中体部分侧压载舱重心为
VCG_SBWT1 = VCG_SBWT2 = VCG_SBWT3 = VCG_SBWT4 = VCG_SBWT5 = VCG_SBWT6 = 9.732
VCG_SBWT7 = 13.417

LCG_SBWT2 = start_cargo2 + L_cargo2 /2
LCG_SBWT3 = start_cargo3 + L_cargo3 /2
LCG_SBWT4 = start_cargo4 + L_cargo4 / 2
LCG_SBWT5 = start_cargo5 + L_cargo5 /2
LCG_SBWT6 = start_cargo6 + L_cargo6 / 2
LCG_SBWT1 = start_cargo1 + 13.289 # 认为长度方向每个位置的宽度一致，则质心纵向坐标就等效为侧面积形心的x坐标
LCG_SBWT7 = start_cargo7 + 12.77 # 认为长度方向每个位置的宽度一致，则质心纵向坐标就等效为侧面积形心的x坐标

TCG_SBWT2 = TCG_SBWT3 = TCG_SBWT4 = TCG_SBWT5 = TCG_SBWT6 = -15.279
TCG_SBWT1 = -12.883
TCG_SBWT7 = -14.856
print("""
第一侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
第二侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
第三侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
第四侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
第五侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
第六侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
第七侧压载舱S的纵向、横向、垂向坐标为  (%.2f, %.2f, %.2f) m3
"""%(LCG_SBWT1,TCG_SBWT1,VCG_SBWT1,
    LCG_SBWT2,TCG_SBWT2,VCG_SBWT2,
    LCG_SBWT3,TCG_SBWT3,VCG_SBWT3,
    LCG_SBWT4,TCG_SBWT4,VCG_SBWT4,
    LCG_SBWT5,TCG_SBWT5,VCG_SBWT5,
    LCG_SBWT6,TCG_SBWT6,VCG_SBWT6,
    LCG_SBWT7,TCG_SBWT7,VCG_SBWT7))


第一侧压载舱S的纵向、横向、垂向坐标为  (192.89, -12.88, 9.73) m3
第二侧压载舱S的纵向、横向、垂向坐标为  (166.42, -15.28, 9.73) m3
第三侧压载舱S的纵向、横向、垂向坐标为  (140.92, -15.28, 9.73) m3
第四侧压载舱S的纵向、横向、垂向坐标为  (115.42, -15.28, 9.73) m3
第五侧压载舱S的纵向、横向、垂向坐标为  (89.92, -15.28, 9.73) m3
第六侧压载舱S的纵向、横向、垂向坐标为  (64.42, -15.28, 9.73) m3
第七侧压载舱S的纵向、横向、垂向坐标为  (40.22, -14.86, 13.42) m3



## 顶压载舱

In [10]:
# 母船顶压载舱容积
V_TBWTP_parent = 710.27
V_TBWTS_parent = 710.35

LCG_TBWTP_ratio_parent = (195.025 - 183.98)/23.49 #m
LCG_TBWTS_ratio_parent = (195.024 - 183.98)/23.49 #m
VCG_TBWTP_ratio_parent = (18.808 - 1.71)/17.89
VCG_TBWTS_ratio_parent = (18.807 - 1.71)/17.89

In [11]:
V_TBWTP = k * V_TBWTP_parent
V_TBWTS = k * V_TBWTS_parent
LCG_TBWTP = start_cargo1 + LCG_TBWTP_ratio_parent * L_cargo1
LCG_TBWTS = start_cargo1 + LCG_TBWTS_ratio_parent * L_cargo1
VCG_TBWTP = start_v_cargo + VCG_TBWTP_ratio_parent * Height_cargo
VCG_TBWTS = start_v_cargo + VCG_TBWTS_ratio_parent * Height_cargo
print("""
第一顶压载舱P的舱容为                 %.2f        m3
第一顶压载舱S的容积为                 %.2f        m3
第一顶压载舱P的重心纵向、横向坐标为  (%.2f, %.2f) m3
第一顶压载舱S的重心纵向、横向坐标为  (%.2f, %.2f) m3
"""%(V_TBWTP,V_TBWTS,LCG_TBWTP,VCG_TBWTP,LCG_TBWTS,VCG_TBWTS))


第一顶压载舱P的舱容为                 682.74        m3
第一顶压载舱S的容积为                 682.82        m3
第一顶压载舱P的重心纵向、横向坐标为  (191.19, 18.56) m3
第一顶压载舱S的重心纵向、横向坐标为  (191.19, 18.56) m3



## 双层底压载舱

In [26]:
from scipy.interpolate import interp1d
from scipy.integrate import simpson
import numpy as np
# 第一双层底压载舱
x = [0,0.5,1,2]
A = [107.142,164.021,188.534,222.39]
xg = [42.29,41.53,41.31,41.03]

A_inter = interp1d(x,A)
xg_inter = interp1d(x,xg)
x_list = np.linspace(0,1.688,100)
# 体积计算
A_list = A_inter(x_list)
V_DBWT_1 = simpson(A_list,dx=1.688/100)
# 形心计算
xg_list = xg_inter(x_list)
zg_list = x_list
LCG_DBWT_1 = np.sum(A_list * xg_list) / xg_list.sum()
VCG_DBWT_1 = np.sum(A_list * zg_list) / zg_list.sum()
print("""
第一压载水舱（左、右）的容积为                %.2f m3
第一压载水舱（左、右）形心的纵向、垂坐标为   (%.2f, %.2f) m3
"""%(V_DBWT_1,LCG_DBWT_1,VCG_DBWT_1))


第一压载水舱（左、右）的容积为                290.72 m3
第一压载水舱（左、右）形心的纵向、垂坐标为   (173.59, 190.28) m3



# 淡水舱

In [13]:
# 母船
V_FWTP_parent = V_FWTS_parent =218.06
V_DWT_parent = 23.52 # 蒸馏水舱

LCG_FWTP_ratio_parent = LCG_FWTS_ratio_parent = 4.256 / 6.75
VCG_FWTP_ratio_parent = VCG_FWTS_ratio_parent = (17.165 - 13.7) / 6.296
LCG_DWT_ratio_parent = (7.5 - 6.75) / 1.5
VCG_DWT_ratio_parent = (14.7 - 13.7) / 2

In [14]:
# 母船缩放
V_FWTP = k * V_FWTP_parent
V_FWTS = k * V_FWTS_parent
V_DWT = k * V_DWT_parent

LCG_FWTP = LCG_FWTS = LCG_FWTP_ratio_parent * 6.77
VCG_FWTP = VCG_FWTS = 13.522 + VCG_FWTP_ratio_parent * 6.074
LCG_DWT = 6.77 + LCG_DWT_ratio_parent * 1.481
VCG_DWT = 13.522 + VCG_DWT_ratio_parent * 1.974
print("""
淡水舱（左）的容积为    %.2f m3
淡水舱（右）的容积为    %.2f m3
蒸馏水舱的容积为        %.2f m3
淡水舱（左）重心的纵向、垂向坐标为    (%.2f, %.2f) m3
淡水舱（右）中心的纵向、垂向坐标为    (%.2f, %.2f) m3
蒸馏水舱中心的纵向、垂向坐标为        (%.2f, %.2f) m3
"""%(V_FWTP,V_FWTS,V_DWT,
     LCG_FWTP,VCG_FWTP,
     LCG_FWTS,VCG_FWTS,
     LCG_DWT,VCG_DWT))


淡水舱（左）的容积为    209.61 m3
淡水舱（右）的容积为    209.61 m3
蒸馏水舱的容积为        22.61 m3
淡水舱（左）重心的纵向、垂向坐标为    (4.27, 16.86) m3
淡水舱（右）中心的纵向、垂向坐标为    (4.27, 16.86) m3
蒸馏水舱中心的纵向、垂向坐标为        (7.51, 14.51) m3



# 燃油舱

## 柴油舱

In [15]:
# 柴油舱设计容量取130m3，大于计算所需的123.39m3
V_doil = 130
# 总布置图中柴油舱分为左右两舱，每舱容积65m3
# 有CAD测量或测质心数据为
x_doil = 25.37
z_doil = 3.86
print(f"""
柴油舱的容积为             {V_doil} m3
质心的纵向、垂向坐标为     ({x_doil},{z_doil}) m3
""")


柴油舱的容积为             130 m3
质心的纵向、垂向坐标为     (25.37,3.86) m3



## 燃油沉积舱

In [16]:
V_depo_oil1 = V_depo_oil2 = 30 # 设计燃油沉降容积为30m3
# 横向布置方面从总布置测量获得
x_depo_oli1 = 20.74 # 第一燃油沉积舱
x_depo_oli2 = 24.69 # 第二燃油沉积舱
z_depo_oli1 = z_depo_oli2 = 16.96
print("""
燃油沉积的容积为                    %.2f m3
第一燃油沉积舱的纵向、垂向位置为    (%.2f,%.2f) m3
第二燃油沉积舱的纵向、垂向位置为    (%.2f,%.2f) m3
"""%(V_depo_oil1+V_depo_oil2,
    x_depo_oli1,z_depo_oli1,
    x_depo_oli2,z_depo_oli2))


燃油沉积的容积为                    60.00 m3
第一燃油沉积舱的纵向、垂向位置为    (20.74,16.96) m3
第二燃油沉积舱的纵向、垂向位置为    (24.69,16.96) m3



## 燃油舱

In [17]:
# 第一燃油舱，位于平行中体范围内，线型基本无收缩
A = 19.95 # 横截面面积: m2
L_oil1 = 51 # m
V_oil1_right = V_oil1_left = A * L_oil1 # m3
x_oil1_left = x_oil1_right = 154 # CAD量取
z_oil_left = z_oil_right = 18.02 # CAD量取
print("""
第一燃油舱（左）的容积为                         %.2f m3
第一燃油舱（左）的形心纵向、垂向位置为          (%.2f, %.2f) m3
第一燃油舱（右）的容积为                         %.2f m3
第一燃油舱（右）的形心纵向、垂向位置为          (%.2f, %.2f) m3
"""%(V_oil1_left,x_oil1_left,z_oil_left,
    V_oil1_right,x_oil1_right,z_oil_right))


第一燃油舱（左）的容积为                         1017.45 m3
第一燃油舱（左）的形心纵向、垂向位置为          (154.00, 18.02) m3
第一燃油舱（右）的容积为                         1017.45 m3
第一燃油舱（右）的形心纵向、垂向位置为          (154.00, 18.02) m3



In [18]:
# 第二燃油舱，位于平行中体范围内，线型基本无收缩
A = 19.95
L_oil2 = 51
V_oil2_left = V_oil2_right = A * L_oil2
x_oil2_left = x_oil2_right = 102.247
z_oil2_left = z_oil2_right = 18.02
print("""
第二燃油舱（左）的容积为                 %.2f m3
第二燃油舱（左）的形心纵向、垂向位置为   (%.2f, %.2f) m3
第二燃油舱（右）的容积为                 %.2f m3
第二燃油舱（右）的形心纵向、垂向位置为   (%.2f, %.2f) m3
"""%(V_oil2_left,x_oil2_left,z_oil2_left,
    V_oil2_right,x_oil2_right,z_oil2_right))


第二燃油舱（左）的容积为                 1017.45 m3
第二燃油舱（左）的形心纵向、垂向位置为   (102.25, 18.02) m3
第二燃油舱（右）的容积为                 1017.45 m3
第二燃油舱（右）的形心纵向、垂向位置为   (102.25, 18.02) m3



In [19]:
# 第三燃油舱 位于 2 - 8站之间
x = [21.517,26.896,32.2755,43.034,53.7925,64.551,75.3095,86.068]
A = [19.81,21.04,20.04,19.95,19.95,19.95,19.95,19.95]
zg = [18,17.95,18.01,18.02,18.02,18.02,18.02,18.02]

A_inter = interp1d(x,A)
zg_inter = interp1d(x,zg)
x_list = np.linspace(27.45,76.75,100)
# 体积计算
A_list = A_inter(x_list)
V_oil3 = simpson(A_list,dx=(76.75-27.45)/100)
# 重心计算
xg_list = x_list
zg_list = zg_inter(x_list)
LCG_oil3 = np.sum(A_list * xg_list) / A_list.sum()
VCG_oil3 = np.sum(A_list * zg_list) / A_list.sum()
print("""
第三燃油舱（左、右）的容积为               %.2f m3
第三燃油舱（左、右）的纵向、垂向坐标为     (%.2f, %.2f)m3
"""%(V_oil3,LCG_oil3,VCG_oil3))


第三燃油舱（左、右）的容积为               976.75 m3
第三燃油舱（左、右）的纵向、垂向坐标为     (52.03, 18.01)m3



In [20]:
# 第四燃油舱，高度9.732-18.358范围内
x = list(range(9,20))
A = [18.69,25.14,30.28,34.48,37.95,40.83,42.99,44.66,45.78,46.45,46.72]
xg = [23.164,22.927,22.871,22.739,22.685,22.65,22.61,22.57,22.55,22.53,22.53]

A_inter = interp1d(x,A)
xg_inter = interp1d(x,xg)
x_list = np.linspace(9.732,18.358,100)
# 体积计算
A_list = A_inter(x_list)
V_oil4 = simpson(A_list,dx=(18.358-9.732)/100)
# 重心计算
xg_list = xg_inter(x_list)
zg_list = x_list
LCG_oil4 = np.sum(A_list * xg_list) / A_list.sum()
VCG_oil4 = np.sum(A_list * zg_list) / A_list.sum()
print("""
第四燃油舱（左、右）的容积为                   %.2f m3
第四燃油舱（左、右）的纵向、垂向坐标为         (%.2f, %.2f) m3
"""%(V_oil4,LCG_oil4,VCG_oil4))


第四燃油舱（左、右）的容积为                   332.56 m3
第四燃油舱（左、右）的纵向、垂向坐标为         (22.66, 14.47) m3



# 污油舱

In [21]:
# 滑油循环舱
L,W,H = 8.686,1.974,0.987
V_HC = L * W * H
LCG_HC = 20.275
VCG_HC = 1.125
# 滑油污油舱
x = [0,0.5,1,2] # 水线高度
xg = [25.394,24.908,24.839,24.77] # 重心x坐标
A = [5.56,12.678,15.66,20.338] # 面积
xg_inter = interp1d(x,xg)
A_inter = interp1d(x,A)
zg_list = np.linspace(0.632,1.619,num=100)
# 体积计算
A_list = A_inter(zg_list)
V_HW = simpson(A_list,dx=(1.619-0.632)/100)
# 重心计算
xg_list = xg_inter(zg_list)
LCG_HW = np.sum(A_list * xg_list) / A_list.sum()
VCG_HW = np.sum(A_list * zg_list) / A_list.sum()
print("""
滑油循环舱的容积为   %.2f m3
滑油循环舱的纵向、垂向坐标为  (%.2f, %.2f) m3
滑油污油舱的容积为   %.2f m3
滑油污油舱的纵向、垂向坐标为  (%.2f, %.2f) m3
"""%(V_HC,LCG_HC,VCG_HC,
     V_HW,LCG_HW,VCG_HW))


滑油循环舱的容积为   16.92 m3
滑油循环舱的纵向、垂向坐标为  (20.27, 1.12) m3
滑油污油舱的容积为   15.79 m3
滑油污油舱的纵向、垂向坐标为  (24.83, 1.15) m3



# 污水和其他

## 舱底水分离舱

In [22]:
z_range = [0,1.619]
zg_list = np.linspace(*z_range,num=100)

x = [0,0.5,1,2]
xg = [18.99,18.906,18.891,18.885]
A = [8.088,15.664,18.935,23.461]
xg_inter = interp1d(x,xg)
A_inter = interp1d(x,A)
# 体积计算
A_list = A_inter(zg_list)
V_BWF = simpson(A_list,dx=(z_range[1] - z_range[0])/100)
# 重心计算
xg_list = xg_inter(zg_list)
LCG_BWF = np.sum(A_list * xg_list) / A_list.sum()
VCG_BWF = np.sum(A_list * zg_list) / A_list.sum()
print("""
舱底水分离舱的容积为     %.2f m3
舱底水分离舱重心的横向坐标为  (%.2f, %.2f) m3
"""%(V_BWF,LCG_BWF,VCG_BWF))


舱底水分离舱的容积为     26.90 m3
舱底水分离舱重心的横向坐标为  (18.90, 0.91) m3



## 燃油污油舱

In [23]:
print("""
燃油污油舱的容积为   %.2f m3
燃油污油舱的纵向、垂向坐标为  (%.2f, %.2f) m3
"""%(V_HW,LCG_HW,VCG_HW))


燃油污油舱的容积为   15.79 m3
燃油污油舱的纵向、垂向坐标为  (24.83, 1.15) m3



# 生活用品

In [24]:
# 获取重量，估计坐标
food_weight = 6.76
spares_weight = 93.27
luggage_weight = 1.26
LCG_food, VCG_food = 13.05,23.14
LCG_spares, VCG_spares = 18.194,23.14
LCG_luggage, VCG_luggate = 16.25,26.2

# 空船重量与重心

In [10]:
# 空船重量为
W_empty_ship = 12498.28
# 重心数据利用母船进行缩放
LCG_empty_ship = 101.10 * Lpp / Lpp_parent # 指导书p94
VCG_empty_ship = 11.66 * D / D_parent
print("""
空船重量为               %.2f t
空船的纵向、横向坐标为  (%.2f, %.2f) m
"""%(W_empty_ship, LCG_empty_ship, VCG_empty_ship))


空船重量为               12498.28 t
空船的纵向、横向坐标为  (99.79, 11.51) m

